In [1]:
!git clone https://github.com/federicozanini/NLP.git

Cloning into 'NLP'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 205 (delta 51), reused 77 (delta 20), pack-reused 87
Receiving objects: 100% (205/205), 319.78 MiB | 18.25 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [2]:
%tensorflow_version 1.15
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_hub as hub
print(tf.__version__)
import os
import numpy as np
import pandas as pd
import json
import spacy
import en_core_web_sm

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


In [3]:
input_dataset = './NLP/utildatas.csv'
df = pd.read_csv(input_dataset)
N_OF_QAS = df.shape[0]
df.head()

,title,context,question,text,answer_start
0,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515
1,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188
2,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279
3,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381
4,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92


In [4]:
# CASTING TO STRING TYPE THE "text" COLUMN
df["text"] = df["text"].apply(str)

# LOWERING THE CASE OF ALL TEXT COLUMN
df["question"] = df["question"].str.lower()
df["text"] = df["text"].str.lower()
df["context"] = df["context"].str.lower()

# get rid of problem chars
df["question"] = df["question"].replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ') #get rid of problem chars
df["text"] = df["text"].replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ') #get rid of problem chars
df["context"] = df["context"].replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ') #get rid of problem chars

# RESULT
df.head()

,title,context,question,text,answer_start
0,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",to whom did the virgin mary allegedly appear i...,saint bernadette soubirous,515
1,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",what is in front of the notre dame main building?,a copper statue of christ,188
2,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",the basilica of the sacred heart at notre dame...,the main building,279
3,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",what is the grotto at notre dame?,a marian place of prayer and reflection,381
4,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",what sits on top of the main building at notre...,a golden statue of the virgin mary,92


In [5]:
# little utils func, find the index of a sequence inside a list 
def find_sublist(context_list, answer_list):
  for i, word in enumerate(context_list):
    if context_list[i:i+len(answer_list)]==answer_list:
      return (i, i+len(answer_list)-1)
  return (-1, -1)

In [6]:
# CREATING THE LISTS FROM THE STRINGS IN THE DF
rows = []
sbagliate = []
counter = 0

for i, row in df.iterrows():
  question_list = row["question"].split(" ") 
  context_list = row["context"].split(" ")
  answer_list = row["text"].split(" ")
  start, end = find_sublist(context_list, answer_list)
  
  new_cols = {
      "question_list" : question_list,
      "context_list" : context_list,
      "answer_list" : answer_list,
      "answer_start" : start,
      "answer_end" : end
  }

  if start == -1:
    sbagliate.append(new_cols)

  rows.append(new_cols)

#ADDING THE NEW COLUMNS
df[["question_list", "context_list", "answer_list", "answer_start", "answer_end"]] = pd.DataFrame(rows)

df.head()

,title,context,question,text,answer_start,question_list,context_list,answer_list,answer_end
0,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",to whom did the virgin mary allegedly appear i...,saint bernadette soubirous,90,"[to, whom, did, the, virgin, mary, allegedly, ...","[architecturally,, the, school, has, a, cathol...","[saint, bernadette, soubirous]",92
1,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",what is in front of the notre dame main building?,a copper statue of christ,32,"[what, is, in, front, of, the, notre, dame, ma...","[architecturally,, the, school, has, a, cathol...","[a, copper, statue, of, christ]",36
2,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",the basilica of the sacred heart at notre dame...,the main building,25,"[the, basilica, of, the, sacred, heart, at, no...","[architecturally,, the, school, has, a, cathol...","[the, main, building]",27
3,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",what is the grotto at notre dame?,a marian place of prayer and reflection,-1,"[what, is, the, grotto, at, notre, dame?]","[architecturally,, the, school, has, a, cathol...","[a, marian, place, of, prayer, and, reflection]",-1
4,University_of_Notre_Dame,"architecturally, the school has a catholic cha...",what sits on top of the main building at notre...,a golden statue of the virgin mary,-1,"[what, sits, on, top, of, the, main, building,...","[architecturally,, the, school, has, a, cathol...","[a, golden, statue, of, the, virgin, mary]",-1


In [7]:
def split_into_sentences(text):
  context_doc = nlp(text)
  context_sentences = []
  for j in context_doc.sents:
    if len(j)>1:
      context_sentences.append(j.string.strip()) #tokenize into sentences
  #print(context_sentences)
  return context_sentences
  

In [ ]:
nlp = spacy.load('en_core_web_sm')
df["context_sentences"] = df['context'].apply(lambda x: split_into_sentences(x))

In [ ]:
df.head(10)

##Importing titles from the test_set

In [ ]:
#Loading the data 
import json
import os


# DATAFRAME ROWS
dataframerows = []

with open('/content/NLP/FINAL_DATASETS/VERSION_IDX_BY_WORD/test_set.json') as f:
    data = json.load(f)

    for el in data['data']:
        title = el['title']
        paragraphs = el['paragraphs']
        
        for context_qas in paragraphs:
              context = context_qas['context'] 
              qas = context_qas['qas']

              for a_q in qas: #[0]["answers"][0]["text"]
                  answer = a_q['answers']
                  question = a_q['question']
                  id = a_q['id']
                  
                  for a in answer:
                      text = a['text']
                      answer_start = a['answer_start']
                      
                      # CREATE ROW
                      row = {
                          #"id" : id,
                          "title" : title,
                          #"paragraphs" : paragraphs,
                          #"context_qas" : context_qas,
                          #"context" : context,
                          #"qas" : qas,
                          #"question" : question,
                          #"answer" : answer,
                          #"text" : text,
                          #"answer_start" : answer_start,
                          }
                      
                      # APPEND ROW TO DATAFRAME ROWS
                      dataframerows.append(row)

    df_test = pd.DataFrame(dataframerows)
    df_test = df_test["title"]]

In [ ]:
df_test.head()

In [ ]:
df_test.shape

##Filter the whole dataset by keeping only the titles in the test set

In [ ]:
test_titles = df_test.title.to_list()

In [ ]:
df = df[df.title.isin(test_titles)]
print(df.shape)
df.head()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
####### CHECKPOINT #######
#df.to_pickle(path="./ELMO_df_test_BEFORE_QA.zip", compression='zip')